In [11]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('max_columns', 100)

In [12]:
mapping_filename        = '../data/gene_names_mapping_new.tsv'
gtex_reads_filename       = '/data5/16p12_RNA/GTEx/data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct'
gtex_tpm_filename       = '/data5/16p12_RNA/GTEx/data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct'
gtex_sample_info_filename = '/data5/16p12_RNA/GTEx/data/GTEx_v7_Annotations_SampleAttributesDS.txt'



In [3]:
mapp   = pd.read_csv(mapping_filename, sep='\t', index_col='ensembl')
gpheno = pd.read_csv(gtex_sample_info_filename, sep='\t')
gpheno = gpheno.set_index('SAMPID', drop=False)
['SMTS', 'SMTSD']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['SMTS', 'SMTSD']

In [4]:
greads = pd.read_csv(gtex_reads_filename, sep='\t', skiprows=2)
greads_backup = greads.copy()

In [5]:
gtpm = pd.read_csv(gtex_tpm_filename, sep='\t', skiprows=2)
gtpm_backup = gtpm.copy()

In [6]:
greads = greads_backup.copy()
gtpm = gtpm_backup.copy()

In [7]:
def strip_gencode(s):
    return s.split('.')[0]
greads = greads.set_index(greads.Name.apply(strip_gencode))
greads = greads.drop('Name', axis=1)
greads = greads.drop('Description', axis=1)

In [8]:
def strip_gencode(s):
    return s.split('.')[0]
gtpm = gtpm.set_index(gtpm.Name.apply(strip_gencode))
gtpm = gtpm.drop('Name', axis=1)
gtpm = gtpm.drop('Description', axis=1)

In [9]:
greads.columns = gpheno.loc[greads.columns, 'SMTSD']
gtpm.columns = gpheno.loc[gtpm.columns, 'SMTSD']

In [10]:
df = mapp.copy()
df = df[[]]
for s in greads.columns.unique():
    df[s] = '.'

In [14]:
# create tpm by tissue

tpm_by_tissue = pd.DataFrame()

for s in list(set(gtpm.columns)):
    tpm_by_tissue[s] = gtpm[s].T.median()

In [17]:
summ = pd.DataFrame(index=tpm_by_tissue.index)
summ['median'] = 0
summ['mean'] = 0
summ['std'] = 0

for gene in summ.index:
    summ.loc[gene, 'median'] = tpm_by_tissue.loc[gene].median()
    summ.loc[gene, 'mean'] = tpm_by_tissue.loc[gene].mean()
    summ.loc[gene, 'std'] = tpm_by_tissue.loc[gene].std()

In [19]:
def is_enriched_in_tissue(tpm, std_gene, med_gene):
    if tpm > med_gene + 2* std_gene:
        return True
    return False


summ['enriched_tissues'] = '.'

for gene in summ.index:
    std_gene = summ.loc[gene, 'std']
    med_gene = summ.loc[gene, 'median']
    enrichements =  tpm_by_tissue.loc[gene].apply(is_enriched_in_tissue, std_gene=std_gene, med_gene=med_gene)
    enrichements = enrichements[enrichements].index.tolist()

    enrichements = ';'.join(enrichements)
    summ.at[gene, 'enriched_tissues'] = enrichements

In [21]:
summ.to_csv('gtex_preferential_tissue_enrichment.tsv', sep='\t', index=True)